In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

crimes = pd.read_csv('fatal-police-shootings-data.csv')

In [ ]:
crimes

In [ ]:
print(crimes.isnull().sum())
crimes['armed'].fillna('U', inplace=True)
crimes['gender'].fillna('U', inplace=True)
crimes['race'].fillna('U', inplace=True)
crimes['flee'].fillna('U', inplace=True)

In [ ]:
#Data cleaning
crimes['date'] = pd.to_datetime(crimes['date'], format='%Y-%m-%d') #changing str to date type
crimes['age'] = crimes['age'].fillna(0).astype(int) #chaning str to int type

#removing ID column, in order to remove missing indexes
#crimes.reset_index()
#crimes.drop(columns=['id'], inplace=True)

Przekształć tabelę w taki sposób, aby wskazywała zestawienie jednocześnie liczby ofiar interwencji według rasy (‘race’) oraz tego, czy wykazywały one oznaki choroby psychicznej (‘signs_of_mental_illness’).

In [ ]:
race_ilness = crimes.pivot_table(values='name', index='race', columns='signs_of_mental_illness', aggfunc=[len],fill_value=0, margins=True)
race_ilness

Za pomocą Map, Applymap lub Apply dodaj do tego zestawienia kolumnę wskazującą jaki odsetek ofiar interwencji wykazywało oznaki choroby psychicznej dla każdej z ras. Odpowiedz, która z nich charakteryzuje się największym odsetkiem znamion choroby psychicznej podczas interwencji.

In [ ]:
def percentage(row):

    number = round((row['len'][True]/(row['len']['All']/100)), 2)
    return number

race_ilness['percentage'] = race_ilness.apply(lambda row: percentage(row),axis=1)
race_ilness

Największy odsetek znamion choroby wykazuje rasa 'Biała'.

Dodaj kolumnę oznaczającą dzień tygodnia, w którym doszło do interwencji. Zlicz interwencje według odpowiedniego dnia tygodnia. Następnie stwórz wykres kolumnowy, tak aby dni tygodnia były uszeregowane od poniedziałku do niedzieli.

In [ ]:
crimes['week_day'] = crimes['date'].dt.day_name()
days = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

day_of_intervention = crimes.groupby('week_day')['date'].count().reindex(days)

day_of_intervention

day_of_intervention.plot.bar(x='week_day', y=None)

Wczytaj do projektu dane dotyczące populacji w poszczególnych stanach USA oraz dane dotyczące skrótów poszczególnych stanów. Połącz te bazy danych w taki sposób, aby móc policzyć do ilu incydentów w bazie dotyczącej śmiertelnych interwencji doszło w przeliczeniu na 1000 mieszkańców każdego ze stanów.

In [ ]:
data_1 = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population', header=0)
data_1[0]
states = data_1[0]

states_population = pd.DataFrame({'State':states['State'], 'Population':states['Population estimate, July 1, 2019[2]']})
states_population.drop(states_population.index[[56,57,58,59]], inplace=True)
states_population

In [ ]:
data_2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations', header=0)
data_2[0]
states_2 = data_2[0]

states_2.columns = range(states_2.columns.size)

states_acronimes = pd.DataFrame({'State':states_2[0], 'Acronime':states_2[5]})
states_acronimes.dropna(axis='index', inplace=True)
states_acronimes

In [ ]:
states_ac_pop = pd.merge(states_population,states_acronimes,how='inner',on='State')
states_ac_pop

In [ ]:
state_3 = crimes.groupby('state')['name'].count()
state_of_interventions = state_3.to_frame(name='#Interventions')
state_of_interventions.reset_index(inplace=True)
state_of_interventions.rename(columns={'state':'Acronime'}, inplace=True)
state_of_interventions

In [ ]:
final_table = pd.merge(state_of_interventions, states_ac_pop, how='inner',on='Acronime')
final_table

In [ ]:
def percentage(row):

    number = row['Population']/1000
    final = round((row['#Interventions']/number), 3)
    return final

final_table['Percentage'] = final_table.apply(lambda row: percentage(row),axis=1)

In [ ]:
final_table